In [2]:
from bertinizer import plot_data, analyze_correlation, find_outliers, dataset_overview_simplified, find_pca_num_only, explode_datetime_col, plot_optimal_k_kmeans
import pandas as pd
from sklearn.cluster import KMeans
import plotly.express as px
import numpy as np
#from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

# concat all dfs

In [10]:
april_df = pd.read_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-apr14.csv')
august_df = pd.read_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-aug14.csv')
july_df = pd.read_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-jul14.csv')
june_df = pd.read_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-jun14.csv')
may_df = pd.read_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-may14.csv')
sept_df = pd.read_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-sep14.csv')

In [11]:
# List of your DataFrames
dfs = [april_df, august_df, july_df, june_df, may_df, sept_df]  # Add more DataFrames as needed

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

In [46]:
dataset_overview_simplified(combined_df)

,Data Type,Non-Null Count,Unique Values,NaN Values
Date/Time,datetime64[ns],4534327,260093,0
Lat,float64,4534327,7092,0
Lon,float64,4534327,11453,0
Base,object,4534327,5,0
Hour,int32,4534327,24,0
Day,int32,4534327,31,0
DayOfWeek,int32,4534327,7,0
DayName,object,4534327,7,0


# create different dfs for time based analysis

In [21]:
combined_df = explode_datetime_col(combined_df, 'Date/Time')

## daily dfs

In [53]:
# Initialize an empty dictionary to hold the DataFrames
days_df = {}

# Loop through each day of the week
for day in range(7):  # 0 is Monday, 6 is Sunday
    # Create a DataFrame for each day and store it in the dictionary with the day name as the key
    days_df[day] = combined_df[combined_df['DayOfWeek'] == day]

# Now you can access each day's DataFrame using the days_df dictionary
mondays_df = days_df[0]
tuesdays_df = days_df[1]
wednesdays_df = days_df[2]
thursdays_df = days_df[3]
fridays_df = days_df[4]
saturdays_df = days_df[5]
sundays_df = days_df[6]

## hourly based dfs 

In [54]:
# Morning DataFrame (7 AM to 12 PM)
morning_df = combined_df[(combined_df['Hour'] >= 7) & (combined_df['Hour'] < 12)]
# Afternoon DataFrame (12 PM to 6 PM)
afternoon_df = combined_df[(combined_df['Hour'] >= 12) & (combined_df['Hour'] < 18)]
# Early Evening DataFrame (6 PM to 9 PM)
early_evening_df = combined_df[(combined_df['Hour'] >= 18) & (combined_df['Hour'] < 21)]
# Late Evening DataFrame (9 PM to 1 AM) - Handling the transition to the next day
late_evening_df = combined_df[(combined_df['Hour'] >= 21) | (combined_df['Hour'] < 1)]
# Night DataFrame (1 AM to 7 AM)
night_df = combined_df[(combined_df['Hour'] >= 1) & (combined_df['Hour'] < 7)]

# save the dfs

In [55]:
# Define a list of day names to use in file names
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Loop through each day of the week and save the corresponding DataFrame
for day in range(7):  # 0 is Monday, 6 is Sunday
    filename = f'/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/db/{day_names[day]}_df.csv'
    days_df[day].to_csv(filename, index=False)

In [56]:
# Save Morning DataFrame (7 AM to 12 PM)
morning_df.to_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/db/morning_df.csv', index=False)

# Save Afternoon DataFrame (12 PM to 6 PM)
afternoon_df.to_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/db/afternoon_df.csv', index=False)

# Save Early Evening DataFrame (6 PM to 9 PM)
early_evening_df.to_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/db/early_evening_df.csv', index=False)

# Save Late Evening DataFrame (9 PM to 1 AM)
late_evening_df.to_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/db/late_evening_df.csv', index=False)

# Save Night DataFrame (1 AM to 7 AM)
night_df.to_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/db/night_df.csv', index=False)


In [57]:
combined_df.to_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/db/combined_df.csv', index=False)

# remove the old dfs

In [58]:
import os

# List of file paths to be deleted
files_to_delete = [
    '/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-apr14.csv',
    '/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-aug14.csv',
    '/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-jul14.csv',
    '/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-jun14.csv',
    '/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-may14.csv',
    '/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-sep14.csv'
]

# Loop through the list and delete each file
for file_path in files_to_delete:
    try:
        os.remove(file_path)
        print(f"Deleted {file_path}")
    except OSError as e:
        print(f"Error: {file_path} : {e.strerror}")

Deleted /Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-apr14.csv
Deleted /Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-aug14.csv
Deleted /Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-jul14.csv
Deleted /Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-jun14.csv
Deleted /Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-may14.csv
Deleted /Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/uber/uber-trip-data/uber-raw-data-sep14.csv
